# Lab 5: 

Lab session by:
* Daniel Hess
* Pandelis Laurens Symeonidis

map from postag to wordnet postag only words adj, nouns and adverbs

Get most frequent wordnet synset
mfs = most frequent wordnet synset is the first synset returned by wn.synset(w, t)
synsets are ordered by frequency, so this is easy, calling the count method on the lemma we can confirm it
lemmas have a count method telling us how many times they are in the corpus (wordnet)

using the synset found 

find the lowest common hypernim, aka least common subsumer, 

normalize similarities!!!!! FIND THE BOUNDS (RESEARCH IT), checking src code
Caviat: depth is precomputed and each tree (telling depth) is for each pos tag, depending on the pos tag the upper/lower bound
simulate-root/create common node/such that they are from the same tree

Which is the best similarity, a similarity metric should meet certain criteria (the best meets all)


In [24]:
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('wordnet_ic')

from nltk.corpus import wordnet_ic, wordnet as wn
from itertools import combinations, product

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\Daniel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


### Penn tag to Wordnet mapping

In [25]:
def convert(word: str, penn_tag: str):

    d = {'NN': 'n', 'NNS': 'n',
          'JJ': 'a', 'JJR': 'a', 'JJS': 'a',
            'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v', 
            'RB': 'r', 'RBR': 'r', 'RBS': 'r'} 
    
    if penn_tag in d: 
        return word, d[penn_tag]
    else:
        return word, None 

### Step 1 Convert penn -> wordnet (POS)

In [26]:
pairs_penn = [('the','DT'), ('man','NN'), ('swim','VB'), ('with', 'PR'), ('a', 'DT'),
('girl','NN'), ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), ('whilst', 'PR'),
('the', 'DT'), ('woman', 'NN'), ('walk', 'VB')]

converted = [convert(w, t) for w, t in pairs_penn]
pairs_wn = [pair for pair in converted if pair[1]] # If pos = None, it is not indexed by wordnet
print(pairs_wn)

[('man', 'n'), ('swim', 'v'), ('girl', 'n'), ('boy', 'n'), ('woman', 'n'), ('walk', 'v')]


### Step 2 compute most frequenst synset for each word and confirm it is the most frequent

In [27]:
synsets = [wn.synsets(lemma=w, pos=t)[0] for w, t in pairs_wn] # Compute most frequent synset AKA synsets[0] as they are ordered by frequency
print(synsets)

# TODO: proffessor asked us to use .count() just to confirm that the synsets[0] is the most frequent (even though it is he wants us to confirm it)'
# Yes he was that vague about it
# Nope idk how to do it

[Synset('man.n.01'), Synset('swim.v.01'), Synset('girl.n.01'), Synset('male_child.n.01'), Synset('woman.n.01'), Synset('walk.v.01')]


### Step 3 Compute the least common hypernim whenever possible
(It is not possible to do it when pos1!=pos2, cant get common hypernim of a verb vs noun)

In [28]:
content = [(w, t, wn.synsets(lemma=w, pos=t)[0]) for w, t, in pairs_wn]

def depth(syn):
    # depth = longest hypernym path length minus 1 (root=0)
    return max(len(p) for p in syn.hypernym_paths()) - 1

results = []
for (w1, pos1, s1), (w2, pos2, s2) in combinations(content, 2):
    if pos1 != pos2:
        continue  # nouns with nouns, verbs with verbs
    lchs = s1.lowest_common_hypernyms(s2)
    lch = max(lchs, key=depth) if lchs else None
    print(f"{w1}[{s1.name()}] ~ {w2}[{s2.name()}] -> LCH: {lch.name()} (depth={depth(lch)})")

man[man.n.01] ~ girl[girl.n.01] -> LCH: adult.n.01 (depth=7)
man[man.n.01] ~ boy[male_child.n.01] -> LCH: male.n.02 (depth=7)
man[man.n.01] ~ woman[woman.n.01] -> LCH: adult.n.01 (depth=7)
swim[swim.v.01] ~ walk[walk.v.01] -> LCH: travel.v.01 (depth=0)
girl[girl.n.01] ~ boy[male_child.n.01] -> LCH: person.n.01 (depth=6)
girl[girl.n.01] ~ woman[woman.n.01] -> LCH: woman.n.01 (depth=8)
boy[male_child.n.01] ~ woman[woman.n.01] -> LCH: person.n.01 (depth=6)


## Another Way of checking best lch, not deepest.

In [29]:
def best_lch(word1, pos1, word2, pos2):
    syns1 = wn.synsets(word1, pos1)
    syns2 = wn.synsets(word2, pos2)
    best = None
    best_score = -1
    for s1, s2 in product(syns1, syns2):
        lchs = s1.lowest_common_hypernyms(s2)
        if not lchs: 
            continue
        lch = max(lchs, key=depth)
        if depth(lch) > best_score:
            best = (s1, s2, lch)
            best_score = depth(lch)
    return best 

for (w1,p1),(w2,p2) in combinations([(w,p) for w,p,_ in content], 2):
    if p1 != p2: 
        continue
    res = best_lch(w1,p1,w2,p2)
    if not res: 
        continue
    s1, s2, lch = res
    print(f"{w1}[{s1.name()}] ~ {w2}[{s2.name()}] -> LCH: {lch.name()} (depth={depth(lch)})")

man[man.n.01] ~ girl[girl.n.01] -> LCH: adult.n.01 (depth=7)
man[man.n.01] ~ boy[boy.n.02] -> LCH: man.n.01 (depth=8)
man[man.n.01] ~ woman[woman.n.01] -> LCH: adult.n.01 (depth=7)
swim[swim.v.01] ~ walk[walk.v.01] -> LCH: travel.v.01 (depth=0)
girl[daughter.n.01] ~ boy[son.n.01] -> LCH: child.n.02 (depth=9)
girl[girl.n.01] ~ woman[woman.n.01] -> LCH: woman.n.01 (depth=8)
boy[boy.n.02] ~ woman[woman.n.01] -> LCH: adult.n.01 (depth=7)


### Step 4 compute similarity between pairs (when possible aka pos1=pos2) using 4 different methods

In [30]:
brown_ic = wordnet_ic.ic('ic-brown.dat')

def fmt(x, nd=3):
    return "—" if x is None else f"{x:.{nd}f}"


for (w1, pos1, s1), (w2, pos2, s2) in combinations(content, 2):
 
    if pos1 != pos2:
        continue

    path = s1.path_similarity(s2)
    try:
        lch = s1.lch_similarity(s2)
    except Exception:
        lch = None
    wup = s1.wup_similarity(s2)
    try:
        lin = s1.lin_similarity(s2, brown_ic) if p1 in ('n', 'v') else None
    except Exception:
        lin = None


    print(f"{w1}[{s1.name()}] ~ {w2}[{s2.name()}]")
    print(f"  path={path:.3f}  lch={lch:.3f}  wup={wup:.3f}  lin={lin:.3f}\n")


man[man.n.01] ~ girl[girl.n.01]
  path=0.250  lch=2.251  wup=0.632  lin=0.714

man[man.n.01] ~ boy[male_child.n.01]
  path=0.333  lch=2.539  wup=0.667  lin=0.729

man[man.n.01] ~ woman[woman.n.01]
  path=0.333  lch=2.539  wup=0.667  lin=0.787

swim[swim.v.01] ~ walk[walk.v.01]
  path=0.333  lch=2.159  wup=0.333  lin=0.491

girl[girl.n.01] ~ boy[male_child.n.01]
  path=0.167  lch=1.846  wup=0.632  lin=0.293

girl[girl.n.01] ~ woman[woman.n.01]
  path=0.500  lch=2.944  wup=0.632  lin=0.907

boy[male_child.n.01] ~ woman[woman.n.01]
  path=0.200  lch=2.028  wup=0.667  lin=0.318



### TODO

- Confirm that synsets[0] is most frequent, by using .count() on the lemma (not very sure what that means)
- Normalize similarities of each method so that they are comparable
- Why does the least common hypernym sometimes not make sense (depending on which sense/synset you use) -> not the same comparing man v.s boy and man v.s male_child
- Analysis

## Analysis